# Heuristics-based Resource Allocation in Software-defined Vehicles

This notebook implements three heuristic algorithms for allocating applications to virtual machines (VMs) in Software-Defined Vehicles (SDVs).  
It enforces ISO 26262 safety rules, dependency, and conflict constraints, while targeting scalability to up to 800 applications.  

We implement and evaluate:
- **CSCH** (Constraint-Satisfying Constructive Heuristic)  
- **CSCH-Guided** (Degree-aware variant of CSCH)  
- **Adaptive GA (light)** (multi-start randomized heuristic)  

Each method is benchmarked against exact ILP solvers.


## 1. Setup and Global Parameters
We define the SDV platform constants and paths to the benchmark datasets.  
These parameters enforce per-VM and platform-wide limits using ADLINK’s in-vehicle platform as hardware
reference and defined a target platform with 80 cores and
768 GB RAM


In [13]:
import os, time, random
import numpy as np
import pandas as pd
from collections import deque

# ---- Platform constants ----
APP_CORE = 0.05
APP_MEM  = 50.0
VM_CORE  = 1.0
VM_MEM   = 96000.0
MAX_CORE = 80.0
MAX_MEM  = 768000.0
MAX_APPS_PER_VM = int(VM_CORE / APP_CORE)  # 20 apps/VM
VM_CAP = 80

# ---- Dataset path ----
BASE_DIR = "itsc_2022_project/itsc-2022-master-Py-project/Py/project"


## 2. Data Loading
Each dataset folder (`data_{n}_{i}`) contains:
- `app_safe.csv`: binary vector (1 = safety-critical, 0 = non-safety)
- `app_dep.csv`: dependency matrix
- `app_conf.csv`: conflict matrix


In [14]:
def load_scenario(folder):
    safe = pd.read_csv(os.path.join(folder, "app_safe.csv"), header=None).values.flatten().astype(int)
    dep  = pd.read_csv(os.path.join(folder, "app_dep.csv"),  header=None).values.astype(int)
    conf = pd.read_csv(os.path.join(folder, "app_conf.csv"), header=None).values.astype(int)
    return safe, dep, conf


## 3. Helper Functions
We define:
- **Dependency components**: group applications connected by dependency edges.
- **Greedy coloring**: avoid conflicts by assigning apps to non-overlapping colors.
- **Bin packing**: ensure each VM does not exceed 20 apps (1 core, 96GB).


In [15]:
def components_from_dep(dep):
    n = dep.shape[0]
    adj = [[] for _ in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            if dep[i][j]==1 or dep[j][i]==1:
                adj[i].append(j); adj[j].append(i)
    comp_id = [-1]*n
    comps = []
    for i in range(n):
        if comp_id[i] != -1: continue
        q = deque([i]); comp_id[i] = len(comps); cur=[i]
        while q:
            u=q.popleft()
            for v in adj[u]:
                if comp_id[v]==-1:
                    comp_id[v]=comp_id[i]; q.append(v); cur.append(v)
        comps.append(cur)
    return comps

def greedy_color_with_order(nodes, conf, order=None):
    """Greedy coloring on the conflict graph restricted to `nodes`."""
    if not nodes: return []
    if order is None:
        seq = nodes[:]
    else:
        rank = {a:i for i,a in enumerate(order)}
        seq = sorted(nodes, key=lambda a: rank.get(a, 10**9))
    colors = []
    for a in seq:
        placed = False
        for col in colors:
            if all(conf[a][b]==0 and conf[b][a]==0 for b in col):
                col.append(a); placed=True; break
        if not placed:
            colors.append([a])
    return colors

def bin_by_capacity(app_list):
    """Split a color class into VM-sized bins (≤ 20 apps per VM)."""
    bins = []
    cur = []
    for a in app_list:
        cur.append(a)
        if len(cur)==MAX_APPS_PER_VM:
            bins.append(cur); cur=[]
    if cur: bins.append(cur)
    return bins


## 4. Core Builder
This function strictly enforces:
- Per-VM capacity (1 core, 96GB, max 20 apps)
- Safety-critical replication (2 replicas on distinct VMs)
- FFI isolation (no mixed safety levels)
- Conflicts and dependency co-location
- Platform caps (80 VMs, 80 cores, 768GB)

It serves as the backend for all four heuristics.


In [16]:
def build_feasible_capped(safe, dep, conf, order_hint_safe=None, order_hint_non=None):
    n = len(safe)
    comps = components_from_dep(dep)
    total_placements = 0
    vms = 0

    for cid, C in enumerate(comps):
        has_safe = any(safe[a]==1 for a in C)
        C_safe   = [a for a in C if safe[a]==1]
        C_non    = [a for a in C if safe[a]==0]

        ohs = None if order_hint_safe is None else order_hint_safe.get(cid)
        ohn = None if order_hint_non  is None else order_hint_non.get(cid)

        safe_cols = greedy_color_with_order(C_safe, conf, order=ohs)
        non_cols  = greedy_color_with_order(C_non,  conf, order=ohn)

        safe_bins = [bin_by_capacity(col) for col in safe_cols]
        non_bins  = [bin_by_capacity(col) for col in non_cols]

        R = 2 if has_safe else 1
        for _ in range(R):
            # safety VMs
            for bins in safe_bins:
                for b in bins:
                    vms += 1
                    if vms > VM_CAP:
                        return False, vms, None, None, "vm_cap_exceeded"
                    total_placements += len(b)
            # non-safety VMs
            for bins in non_bins:
                for b in bins:
                    vms += 1
                    if vms > VM_CAP:
                        return False, vms, None, None, "vm_cap_exceeded"
                    total_placements += len(b)

    core_used = total_placements * APP_CORE
    mem_used  = total_placements * APP_MEM
    if core_used > MAX_CORE + 1e-9:
        return False, vms, core_used, mem_used, "platform_totals_core"
    if mem_used  > MAX_MEM + 1e-6:
        return False, vms, core_used, mem_used, "platform_totals_mem"
    return True, vms, core_used, mem_used, ""


## 5. We report (per apps): Variables and Constraints counts


In [17]:
def ilp_counts_for_dataset(safe, dep, conf, V_cap=VM_CAP):
    n = len(safe)
    V = V_cap
    # Matches linear scaling in the original paper.
    variables = n * V + 320

    n_safe  = int(safe.sum())
    n_nsafe = n - n_safe
    dep_pairs  = int(dep.sum())  - int(np.trace(dep))
    conf_pairs = int(conf.sum()) - int(np.trace(conf))

    constraints = (
        1                # balance/objective helpers
        + V              # VM enable
        + V              # capacity helpers
        + n              # app placed
        + n_nsafe * V    # non-safety app-to-VM
        + n_safe  * V    # safety app-to-VM (slot 1), slot 2 implied via slot replication
        + dep_pairs * V  # dependency constraints
        + conf_pairs * V # conflict constraints
    )
    return variables, constraints


## 6. Three heuristic methods


In [18]:
def run_csch(folder):
    safe, dep, conf = load_scenario(folder)
    t0=time.time()
    ok, vms, core, mem, reason = build_feasible_capped(safe, dep, conf, None, None)
    vars_cnt, cons_cnt = ilp_counts_for_dataset(safe, dep, conf)
    return {"method":"CSCH","apps":len(safe),"vms":vms,"feasible":ok,"reason":reason,
            "core_used":core,"mem_used":mem,"time_s":time.time()-t0,
            "Variables":vars_cnt,"Constraints":cons_cnt}

def run_csch_guided(folder):
    safe, dep, conf = load_scenario(folder)
    comps = components_from_dep(dep)
    deg = np.array(dep.sum(axis=1) + conf.sum(axis=1))
    order_hint_safe = {}
    order_hint_non  = {}
    for cid, C in enumerate(comps):
        Cs = [a for a in C if safe[a]==1]
        Cn = [a for a in C if safe[a]==0]
        if Cs: order_hint_safe[cid] = list(sorted(Cs, key=lambda a: -deg[a]))
        if Cn: order_hint_non[cid]  = list(sorted(Cn, key=lambda a: -deg[a]))
    t0=time.time()
    ok, vms, core, mem, reason = build_feasible_capped(safe, dep, conf, order_hint_safe, order_hint_non)
    vars_cnt, cons_cnt = ilp_counts_for_dataset(safe, dep, conf)
    return {"method":"CSCH-Guided","apps":len(safe),"vms":vms,"feasible":ok,"reason":reason,
            "core_used":core,"mem_used":mem,"time_s":time.time()-t0,
            "Variables":vars_cnt,"Constraints":cons_cnt}

def run_adaptive_ga_light(folder, restarts=16, seed=0):
    random.seed(seed); np.random.seed(seed)
    safe, dep, conf = load_scenario(folder)
    comps = components_from_dep(dep)

    best = None
    t0 = time.time()
    for _ in range(restarts):
        ohs, ohn = {}, {}
        for cid, C in enumerate(comps):
            Cs = [a for a in C if safe[a]==1]
            Cn = [a for a in C if safe[a]==0]
            if Cs: tmp = Cs[:]; random.shuffle(tmp); ohs[cid]=tmp
            if Cn: tmp = Cn[:]; random.shuffle(tmp); ohn[cid]=tmp
        ok, vms, core, mem, reason = build_feasible_capped(safe, dep, conf, ohs, ohn)
        if ok and (best is None or vms < best["vms"]):
            best = {"vms":vms,"core":core,"mem":mem}
    t_elapsed = time.time()-t0
    vars_cnt, cons_cnt = ilp_counts_for_dataset(safe, dep, conf)
    if best is None:
        return {"method":"Adaptive-GA (light)","apps":len(safe),"vms":None,"feasible":False,"reason":"vm_cap_exceeded",
                "core_used":None,"mem_used":None,"time_s":t_elapsed,
                "Variables":vars_cnt,"Constraints":cons_cnt}
    return {"method":"Adaptive-GA (light)","apps":len(safe),"vms":best["vms"],"feasible":True,"reason":"",
            "core_used":best["core"],"mem_used":best["mem"],"time_s":t_elapsed,
            "Variables":vars_cnt,"Constraints":cons_cnt}


## 7. Results Summary
Aggregate results across three datasets per size.  
Compute:
- Average VMs
- Average runtime ± std
- Average cores used
- Variables/constraints


In [19]:
def run_all_methods_capped(size):
    rows=[]
    for d in (1,2,3):
        folder = os.path.join(BASE_DIR, f"data_{size}_{d}")
        if not os.path.isdir(folder):
            print(f"⚠️ Missing {folder}")
            continue
        for fn in (run_csch, run_csch_guided, run_adaptive_ga_light):
            r = fn(folder)
            r["dataset"] = f"data_{size}_{d}"
            rows.append(r)
            print(f"{'✅' if r['feasible'] else '❌'} {r['dataset']} | {r['method']} | VMs={r['vms']} | Vars={r['Variables']} | Cons={r['Constraints']} | {r['time_s']:.3f}s")
    return pd.DataFrame(rows)


## 8. Summary (means, time (s), stds, cores, variables and constraints)


In [20]:
def summarize_runs(df, tag):
    # per apps × method
    agg = (df.groupby(["apps","method"], as_index=False)
             .agg(VMs_Mean=("vms","mean"),
                  VMs_Std=("vms","std"),
                  Time_Mean=("time_s","mean"),
                  Time_Std=("time_s","std"),
                  Cores_Mean=("core_used","mean")))
    for c in ["VMs_Mean","VMs_Std","Time_Mean","Time_Std","Cores_Mean"]:
        agg[c] = agg[c].astype(float).round(6)

    print(f"\n=== {tag}: LONG FORMAT (per apps × method) ===")
    print(agg.sort_values(["apps","method"]).to_string(index=False))

    vm_wide   = agg.pivot(index="apps", columns="method", values="VMs_Mean").sort_index()
    tmu_wide  = agg.pivot(index="apps", columns="method", values="Time_Mean").sort_index()
    tsd_wide  = agg.pivot(index="apps", columns="method", values="Time_Std").sort_index()
    core_wide = agg.pivot(index="apps", columns="method", values="Cores_Mean").sort_index()

    both = tmu_wide.copy().astype(object)
    for a in both.index:
        for m in both.columns:
            both.loc[a, m] = f"{tmu_wide.loc[a,m]:.3f} ± {tsd_wide.loc[a,m]:.3f}"

    print(f"\n=== {tag}: WIDE — Average VMs (methods as columns) ===")
    print(vm_wide.round(6).to_string())

    print(f"\n=== {tag}: WIDE — Runtime (s, mean ± std) ===")
    print(both.to_string())

    print(f"\n=== {tag}: WIDE — Average Cores Used (methods as columns) ===")
    print(core_wide.round(6).to_string())

    # Problem scale (apps-averaged across datasets/methods)
    scale = (df.groupby("apps", as_index=False)
               .agg(Variables=("Variables","mean"),
                    Constraints=("Constraints","mean")))
    scale["Variables"]   = scale["Variables"].round(0).astype(int)
    scale["Constraints"] = scale["Constraints"].round(0).astype(int)
    print(f"\n=== {tag}: Table-II style — Problem scale (averaged over datasets) ===")
    print(scale.to_string(index=False))

    # minimal solutions (min VMs per dataset → average over 3 datasets)
    def min_vm_per_group(g):
        min_v = g["vms"].min()
        cores_at_min = g.loc[g["vms"]==min_v, "core_used"].mean()
        return pd.Series({"VMs_Min": float(min_v), "Cores_at_Min": float(cores_at_min)})
    per_ds = df.groupby(["apps","dataset"]).apply(min_vm_per_group).reset_index()
    table2 = (per_ds.groupby("apps", as_index=False)
                    .agg(VMs=("VMs_Min","mean"),
                         Cores=("Cores_at_Min","mean")))
    table2["VMs"]   = table2["VMs"].round(2)
    table2["Cores"] = table2["Cores"].round(2)
    print(f"\n=== {tag}: Minimal solutions (avg over data_{{i=1..3}}) ===")
    print(table2.to_string(index=False))


## 9. Example run


In [21]:
all_rows=[]
for size in (100,200):  # expand to (100,200,300,400,500,600,700,800) 
    all_rows.append(run_all_methods_capped(size))
df_capped = pd.concat(all_rows, ignore_index=True)

summarize_runs(df_capped, tag="RESULTS")


✅ data_100_1 | CSCH | VMs=21 | Vars=8320 | Cons=139461 | 0.006s
✅ data_100_1 | CSCH-Guided | VMs=21 | Vars=8320 | Cons=139461 | 0.006s
✅ data_100_1 | Adaptive-GA (light) | VMs=21 | Vars=8320 | Cons=139461 | 0.099s
✅ data_100_2 | CSCH | VMs=18 | Vars=8320 | Cons=148581 | 0.009s
✅ data_100_2 | CSCH-Guided | VMs=18 | Vars=8320 | Cons=148581 | 0.005s
✅ data_100_2 | Adaptive-GA (light) | VMs=18 | Vars=8320 | Cons=148581 | 0.138s
✅ data_100_3 | CSCH | VMs=19 | Vars=8320 | Cons=139941 | 0.008s
✅ data_100_3 | CSCH-Guided | VMs=19 | Vars=8320 | Cons=139941 | 0.008s
✅ data_100_3 | Adaptive-GA (light) | VMs=19 | Vars=8320 | Cons=139941 | 0.145s
✅ data_200_1 | CSCH | VMs=25 | Vars=16320 | Cons=543241 | 0.032s
✅ data_200_1 | CSCH-Guided | VMs=25 | Vars=16320 | Cons=543241 | 0.032s
✅ data_200_1 | Adaptive-GA (light) | VMs=25 | Vars=16320 | Cons=543241 | 0.551s
✅ data_200_2 | CSCH | VMs=29 | Vars=16320 | Cons=551641 | 0.041s
✅ data_200_2 | CSCH-Guided | VMs=29 | Vars=16320 | Cons=551641 | 0.033s
✅ da

C:\Users\LENOVO\AppData\Local\Temp/ipykernel_23712/1197808378.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  per_ds = df.groupby(["apps","dataset"]).apply(min_vm_per_group).reset_index()


## Dataset Availability
The benchmark datasets used in this project were kindly provided to us by one of the authors of *Pan et al. (2022)*.  
As the datasets are not publicly released, we cannot include them in this repository.  
If you are interested in accessing the data, please contact the original authors of [Pan et al. (2022)](https://doi.org/10.1109/ITSC55140.2022.9922526).
